# Summarize escape across all sera

In [1]:
import altair as alt

import pandas as pd

The next cell is tagged as `parameters` for `papermill` parameterization:

In [ ]:
site_numbering_map_csv = None
func_effects_csv = None
sera = None